In [30]:
import numpy as np
import matplotlib.pyplot as plt

In [95]:
# 文件位置
file_path = r'D:\wechat_file\WeChat Files\wxid_v697m3pg3x0d22\FileStorage\TempFromPhone\final300_out.xyz'

#所有元素的名称 字典是用来计数这个元素在A附近出现的次数 如果这个初始值为-1则表示不计算A与此元素的近邻
all_element_dic = { 'O':0}

#需要分析的元素以及第一近邻的距离
A_dis = 2.3
name_A = 'Al'

# 分析的起始帧、终止帧、以及步长
start_frame = 0
final_frame = 1
steps_frame = 1

# 是否创建新帧 
creat_new_frame = 0
# 将分析结果存入文件
#save_path = r'C:\Users\zhang\Desktop\40sap_out\\'
#save_name = file_name[:5] + '_ana'

In [96]:
# 现在把文件读取
with open(file_path) as f:
    str_data = f.read()
all_data = str_data.split()
# 选择要处理的帧数
def get_data(frame_num):
    atoms_num = int(all_data[0])
    frame_len = atoms_num*4+2
    data_len =len(all_data)
    max_frame = data_len // frame_len

    temp = []
    temp = all_data[frame_len*frame_num:frame_len*(frame_num+1)]

    #暂时先自行处理了表头
    temp = temp[2:]
    frame_list = []
    for i in range(atoms_num):
        frame_list.append([temp[i*4+0],float(temp[i*4+1]),float(temp[i*4+2]),float(temp[i*4+3])])
    
    return frame_list
#定义距离函数
def cal_distance(x1,y1,z1,x2,y2,z2):
    dis = ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**(1/2)
    return dis

def cal_ppp_distance(pos1, pos2):
    global box_size
    # 计算两个位置之间的最小距离，考虑周期性边界条件
    pos1,pos2,box_size = np.array(pos1),np.array(pos2),np.array(box_size)
    # 计算位置差矢量
    delta = pos2 - pos1
    
    # 应用最小图像约束，确保距离在周期内最短
    delta = delta - np.round(delta / box_size) * box_size
    # 计算距离
    distance = np.linalg.norm(delta)
    if distance<1e-4:
        distance = 100.0
    return distance

In [97]:
box_max_value = 0
for atom in get_data(0):
    max_v = max(atom[1:])
    if max_v > box_max_value:
        box_max_value = max_v
box_size = [box_max_value,box_max_value,box_max_value]
print(box_size)

[51.6558, 51.6558, 51.6558]


In [98]:
data0 = get_data(0)
data0[9900]


['O', 37.7796, 25.4879, 22.7556]

In [99]:
for i in data0:
    if i[0]==name_A:
        print(i)

['Al', 46.1644, 34.8192, 11.6575]
['Al', 7.98596, 50.3578, 28.9833]
['Al', -0.119499, 31.9, 27.6053]
['Al', 50.7372, 40.1003, 34.3746]
['Al', 5.62036, 14.4312, 47.6228]
['Al', 15.6361, 51.2145, 7.40269]
['Al', 44.9365, 21.2217, 40.9901]
['Al', 18.259, 44.0847, 18.2054]
['Al', 21.3167, 11.5652, 25.4387]
['Al', 50.8611, 2.97802, 15.0286]
['Al', 43.0846, 35.3753, 12.2195]
['Al', 46.705, 17.1545, 41.5007]
['Al', 24.0307, 4.41053, 19.7896]
['Al', 0.159305, 24.967, 17.4053]
['Al', 40.4822, 25.7081, 3.51659]
['Al', 26.1148, 33.1587, 38.9159]
['Al', 8.24747, 10.4954, 50.8776]
['Al', 16.0224, 4.60518, 18.9858]
['Al', 27.8442, 40.0428, 7.39325]
['Al', 18.2558, 19.8446, 44.3982]
['Al', 4.71356, 28.0234, 29.5119]
['Al', 22.3498, 35.4399, 6.90696]
['Al', 36.1221, 26.2502, 22.6711]
['Al', 25.8754, 26.2874, 25.9902]
['Al', 22.4586, 39.9758, 43.3156]
['Al', 10.3779, 45.5561, 15.8149]
['Al', 11.7107, 40.0057, 27.7976]
['Al', 49.092, 2.02727, 19.2937]
['Al', 41.9083, 24.0371, 17.4321]
['Al', 38.3999, 7.

In [104]:
# # 记录每一个帧数处理后得到的信息，然后加和
# 得到元素A的数量
num_A_atom = 0
# 记录A总的近邻和Cn数
sum_A_neibor = 0
sum_A_Cn = []
for i in range(50):
    sum_A_Cn.append(0)
# 字典是用来计数的
sum_element_dic = all_element_dic.copy()
# 循环次数
count_cycle = 0
################### 开始循环处理每一帧 #################
for step in range(start_frame,final_frame,steps_frame):
    count_cycle+=1
    # 获取数据
    data = get_data(step)
    # 找到最大的盒子边界以及A元素的列表
    box_length = 0
    for i in data:
        if i[2]>box_length:
            box_length=float(i[2])
            # print(box_length)
    A_list = []
    for i in data:
        if i[0]== name_A:
            if A_dis <i[1]<box_length-A_dis and A_dis <i[2]<box_length-A_dis and A_dis <i[3]<box_length-A_dis :
                A_list.append(i)
    # 记录A元素的总个数
    num_A_atom = len(A_list)
    
    # 计量本次A总的近邻原子次数 后边好计算
    count_A_neibor = 0
    
    # 记录每个A的Cn的分布
    A_cn_list = []
    for i in range(50):
        A_cn_list.append(0)

    # 每一次要拷贝一次字典 便于重新计数
    frame_element_dic = all_element_dic.copy()
    # 开始计算
    for i in A_list:
        # 记录这个元素有几个配位
        A_cn = 0
        for j in data:
            # 如果就是A本身或者与dic中表示了不计数，或者根本没在dic中，那么跳过
            if j[0] not in all_element_dic or all_element_dic[j[0]]== -1 :
                continue
            # 在近邻内
            if cal_distance(i[1],i[2],i[3],j[1],j[2],j[3])<A_dis :
                # 首先记录多了一个近邻
                A_cn+=1
                # 再记录这个近邻的种类 先取出原有值，再加一
                temp = frame_element_dic[j[0]]
                frame_element_dic[j[0]] = temp+1
        # 本循环结束 记录下这个A元素的Cn值        
        A_cn_list[A_cn]+=1
    ####################已经得到了这一帧的所有数据########################   
    # 第一项 Cn累加
    for i in range(50):
        sum_A_Cn[i]+=A_cn_list[i]
    # 第二项 总的配位数累加
    for i in frame_element_dic:
        # 如果表明不计数 则或者就没有考虑进去 就不计算
        if frame_element_dic[i] == -1  :
            continue
        sum_element_dic[i] += frame_element_dic[i]
        

########  计算完成 输出结果  ##############
print('我们关注的是 '+ name_A +' 的配位数，采取的截断距离为 '+str(A_dis))
print(name_A +'的Cn分布如下：')
for i in range(50):
    if sum_A_Cn[i] != 0 :
        print('C' +str(i)+': '+str(sum_A_Cn[i]//count_cycle))
# 再来分析A的平均配位数
sum_cn = 0
for i in sum_element_dic:
    if sum_element_dic[i] != -1:
        sum_cn += sum_element_dic[i]
print(name_A+'的平均Cn是： '+str(round(sum_cn/(num_A_atom*count_cycle),2)))


# 分析各个元素的配位占比
print('元素列表中的总体配位情况如下：')
for i in sum_element_dic:
    if sum_element_dic[i]== -1:
        print(i + " 元素不计入配位计算！")
    else:
        print(i + ' : '+'配位数量：'+str(sum_element_dic[i]//count_cycle)+' 占比： '+ str(round(sum_element_dic[i]/sum_cn*100,1))+'%')

我们关注的是 Al 的配位数，采取的截断距离为 2.3
Al的Cn分布如下：
C3: 21
C4: 501
C5: 3
Al的平均Cn是： 3.97
元素列表中的总体配位情况如下：
O : 配位数量：2082 占比： 100.0%


In [103]:
box_length

51.442

In [60]:
for i in range(0,0,1):
    pass

In [ ]:
(37.8+32.2+32.6)/3

In [ ]:
15.43/3.23